In [2]:
# Load in necessary libraries
import pandas as pd
import numpy as np
import random
import math
import sklearn
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

In [3]:
random.seed(20602629)
#----- Classification Models -----
##Helper functions Base Models##
#Logistic regression returns AUC score
def log_reg(train, test, y_train, y_test, reg='l1', opt = "saga"):
    #If using elasticnet need a l1 ratio - setting to 0.5
    if reg == "elasticnet":
        logisticRegr = LogisticRegression(penalty = reg, solver = opt, max_iter = 100, l1_ratio = 0.5)
    else:
        logisticRegr = LogisticRegression(penalty = reg, solver = opt, max_iter = 100)
    logisticRegr.fit(train, y_train)
    prob_y = logisticRegr.predict_proba(test)
    pred_y = prob_y[:,1]
    return roc_auc_score(y_test, pred_y)

#SVM function returns AUC score
def linSVM(train, test, y_train, y_test, kern = 'rbf'):
    classifier = SVC(kernel = kern, probability = True, max_iter=100)
    classifier.fit(train, y_train)
    prob_y = classifier.predict_proba(test)
    pred_y = prob_y[:,1]
    return roc_auc_score(y_test, pred_y)

#Naive Bayes
def nbayes(train, test, y_train, y_test, laplace = 1e-9):
    classifier = GaussianNB(var_smoothing = laplace)
    classifier.fit(train, y_train)
    prob_y = classifier.predict_proba(test)
    pred_y = prob_y[:,1]
    return roc_auc_score(y_test, pred_y)

#KNN
def knn(train, test, y_train, y_test, num_n = 10):
    classifier = KNeighborsClassifier(n_neighbors=num_n)
    classifier.fit(train, y_train)
    prob_y = classifier.predict_proba(test)
    pred_y = prob_y[:,1]
    return roc_auc_score(y_test, pred_y)

#Decision Trees
def dtree(train, test, y_train, y_test, depth = 5):
    classifier = DecisionTreeClassifier(max_depth = depth)
    classifier.fit(train, y_train)
    prob_y = classifier.predict_proba(test)
    pred_y = prob_y[:,1]
    return roc_auc_score(y_test, pred_y)
  
## Helper functions for base model comparison##
#Run each model with dif parameters on input data - return best validated AUC and index
def comp_LR(X_train, X_test, y_train, y_test):
    #Logistic regression with L1, L2, and elasticnet
    AUC_LR = []
    AUC_LR.append(log_reg(X_train, X_test, y_train, y_test))
    AUC_LR.append(log_reg(X_train, X_test, y_train, y_test, reg="l2"))
    AUC_LR.append(log_reg(X_train, X_test, y_train, y_test, reg="elasticnet"))
    #L1 can use liblinear or saga
    AUC_LR.append(log_reg(X_train, X_test, y_train, y_test, opt="liblinear"))
    #Get max and position of max to find best logistic regression model
    best = np.max(AUC_LR)
    best_index = np.argmax(AUC_LR)
    print("Best LR model Index: {}, AUC: {}".format(best_index, best))

def comp_SVM(X_train, X_test, y_train, y_test):
    #SVM with different kernels
    AUC_SVM = []
    AUC_SVM.append(linSVM(X_train, X_test, y_train, y_test, kern="linear"))
    AUC_SVM.append(linSVM(X_train, X_test, y_train, y_test))
    AUC_SVM.append(linSVM(X_train, X_test, y_train, y_test, kern="poly"))
    AUC_SVM.append(linSVM(X_train, X_test, y_train, y_test, kern="sigmoid"))
    #Get max and position of max to find best model
    best = np.max(AUC_SVM)
    best_index = np.argmax(AUC_SVM)
    print("Best SVM model Index: {}, AUC: {}".format(best_index, best))

def comp_NB(X_train, X_test, y_train, y_test):
    #Naive Bayes with different smoothing parameters
    AUC_NB = []
    AUC_NB.append(nbayes(X_train, X_test, y_train, y_test))
    AUC_NB.append(nbayes(X_train, X_test, y_train, y_test, laplace=1e-5))
    AUC_NB.append(nbayes(X_train, X_test, y_train, y_test, laplace=1e-1))
    AUC_NB.append(nbayes(X_train, X_test, y_train, y_test, laplace=1))
    #Get max and position of max to find best model
    best = np.max(AUC_NB)
    best_index = np.argmax(AUC_NB)
    print("Best NB model Index: {}, AUC: {}".format(best_index, best))

def comp_KNN(X_train, X_test, y_train, y_test):
    #KNN with different num of neighbours
    AUC_KNN = []
    AUC_KNN.append(knn(X_train, X_test, y_train, y_test, num_n=5))
    AUC_KNN.append(knn(X_train, X_test, y_train, y_test, num_n=10))
    AUC_KNN.append(knn(X_train, X_test, y_train, y_test, num_n=15))
    AUC_KNN.append(knn(X_train, X_test, y_train, y_test, num_n=20))
    #Get max and position of max to find best model
    best = np.max(AUC_KNN)
    best_index = np.argmax(AUC_KNN)
    print("Best KNN model Index: {}, AUC: {}".format(best_index, best))

def comp_dtree(X_train, X_test, y_train, y_test):
    #Decision trees with different depths
    AUC_DT = []
    AUC_DT.append(dtree(X_train, X_test, y_train, y_test))
    AUC_DT.append(dtree(X_train, X_test, y_train, y_test, depth=10))
    AUC_DT.append(dtree(X_train, X_test, y_train, y_test, depth=15))
    AUC_DT.append(dtree(X_train, X_test, y_train, y_test, depth=20))
    #Get max and position of max to find best model
    best = np.max(AUC_DT)
    best_index = np.argmax(AUC_DT)
    print("Best DTree model Index: {}, AUC: {}".format(best_index, best))

In [43]:
## Helper functions for Ensemble ##
#Random forest with num estimators and depth
def rforest(train, test, y_train, y_test, n_est = 500, depth=5):
    classifier = RandomForestClassifier(n_estimators = n_est, max_depth=depth)
    classifier.fit(train, y_train)
    prob_y = classifier.predict_proba(test)
    pred_y = prob_y[:,1]
    return roc_auc_score(y_test, pred_y)
#Adaboost with default decision trees and max_depth 1
def ada(train, test, y_train, y_test, n_est = 100):
    ada_class = AdaBoostClassifier(n_estimators=n_est)
    ada_class.fit(train, y_train)
    prob_y = ada_class.predict_proba(test)
    pred_y = prob_y[:,1]
    return roc_auc_score(y_test, pred_y)
#Stacking method with LR used to aggregrate final predictions
def stack(train, test, y_train, y_test, ests = 0):
    if ests == 0:
        ests = [('rf', RandomForestClassifier(n_estimators=100, max_depth = 10)),
        ('ada', AdaBoostClassifier(n_estimators=100)), 
        ('lg', LogisticRegression(solver='liblinear'))]
    stack_class = StackingClassifier(estimators = ests)
    stack_class.fit(train, y_train)
    prob_y = stack_class.predict_proba(test)
    pred_y = prob_y[:,1]
    print("AUC of stacked: {}".format(roc_auc_score(y_test, pred_y)))

#Random Forest tuning
def comp_rf(X_train, X_test, y_train, y_test):
    #RF with different number of estimators - 10-200 and depth
    num_ests = list(range(10, 201, 10))
    depths = list(range(1, 10, 1))
    max_AUC = 0
    for ii in num_ests:
        for dep in depths:
            rf_AUC = rforest(X_train, X_test, y_train, y_test, n_est=ii, depth=dep)
            if rf_AUC > max_AUC:
                max_AUC = rf_AUC
                (depth, est) = (dep, ii)
    print("Best RF Num_Est: {}, depth:{},  AUC: {}".format(est, dep, max_AUC))

#Adaboost tuning
def comp_ada(X_train, X_test, y_train, y_test):
    #Adaboost with different number of estimators - 10-200
    max_AUC = 0
    max_i = 0
    num_ests = list(range(10, 201, 10))
    for ii in num_ests:
        ada_AUC = ada(X_train, X_test, y_train, y_test, n_est=ii)
        if ada_AUC > max_AUC:
            max_AUC = ada_AUC
            max_i = ii
    print("Best Adaboost Num_Est: {}, AUC: {}".format(num_ests.index(max_i), max_AUC))


#Run simple one layer with different optimization functions to find best
def NN_norm_opt(train, test, y_train, y_test, hidden = 10):
    opts = ["sgd", "adam", "adadelta", "rmsprop"]
    best = 0
    (_,dim) = train.shape
    for opt in opts:
        model = Sequential()
        model.add(Dense(hidden, input_dim=dim, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        #Compile model and fit it
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        model.fit(train, y_train, epochs=5, verbose=0, validation_split=0.2, shuffle=True)
        pred_y = model.predict(test)
        NN_AUC = roc_auc_score(y_test, pred_y)
        if best < NN_AUC:
            best=NN_AUC
            best_opt=opt
    #At end return best AUC and best opts
    return(best, best_opt)

#Functions for 2 and 3 layer NN
def layer2(train, test, n=50, opt="sgd", epoch=50, batch=100):
    # create model
    (_,dim) = train.shape
    model = Sequential()
    model.add(Dense(n, input_dim=dim, activation='relu'))
    model.add(Dense(n, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    #Fit model and run cross-validation
    model.fit(train, test, epochs = epoch, verbose = 0, validation_split = 0.2, shuffle = True, batch_size = batch)
    return model

def layer3(train, test, n=50, opt="sgd", epoch=50, batch=100):
    # create model
    (_,dim) = train.shape
    model = Sequential()
    model.add(Dense(n, input_dim=dim, activation='relu'))
    model.add(Dense(n, activation='relu'))
    model.add(Dense(n, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    #Fit model and run cross-validation
    model.fit(train, test, epochs = epoch, verbose = 0, validation_split = 0.2, shuffle = True, batch_size = batch)
    return model

#NN using above functions with built-in grid search for batch and num neurons
def NN_valid(train, test, y_train, y_test, opt="sgd"):
    #Check 30 - 150 for num neurons by layer and batches 10 - 25
    num_neur = list(range(30, 180, 30))
    batches = [10, 15, 20, 25]
    best_2 = 0
    best_3 = 0
    for batch in batches:
        for n in num_neur:
            #Using batch and neurons get 2 and 3 layer
            model1 = layer2(train, y_train, n=n, batch=batch, opt=opt)
            model2 = layer3(train, y_train, n=n, batch=batch, opt=opt)
            pred_2 = model1.predict(test, batch_size = batch)
            pred_3 = model2.predict(test, batch_size = batch)
            lay2_AUC = roc_auc_score(y_test, pred_2)
            lay3_AUC = roc_auc_score(y_test, pred_3)
            #Update best for each layer with the neuron and batch
            if lay2_AUC > best_2:
                best_2 = lay2_AUC
                (neur_2, batch_2) = (n, batch)
            if lay3_AUC > best_3:
                best_3 = lay3_AUC
                (neur_3, batch_3) = (n, batch)
    #At end check which best is better and return
    if best_2 >= best_3:
      print("Best NN Batch: {}, Neurons: {}, Layers: {}, AUC: {}".format(batch_2, neur_2, "2", best_2))
    else:
      print("Best NN Batch: {}, Neurons: {}, Layers: {}, AUC: {}".format(batch_3, neur_3, "3", best_3))

In [25]:
## Read in data and get train test splits##
train = pd.read_csv('cleaned_train.csv')
test = pd.read_csv('cleaned_test.csv')
train_labels = pd.read_csv('Train_Label.csv')
test_id = pd.read_csv('test.csv')

train.drop('Unnamed: 0', axis = 1, inplace=True)
test.drop('Unnamed: 0', axis = 1, inplace=True)
train_labels.drop('Unnamed: 0', axis = 1, inplace=True)
test_id.drop('Unnamed: 0', axis = 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,6,10,12,21,24,35,85,87,88,89,100,124,126,127,128,130,131,133,166,172,177,193,194,195,205,206,207,220,222,228,229,230,240,257,258,260,261,262) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
test_id

0       14061
1       16467
2       25725
3        9100
4       32597
        ...  
9240    31430
9241     3876
9242    20710
9243    26015
9244    12970
Name: id, Length: 9245, dtype: int64

In [21]:
col_names = list(train.columns)

#Apply normalization before split so it carries through
mm_scaler = MinMaxScaler()
train_mm = mm_scaler.fit_transform(train)
train = pd.DataFrame(train_mm, columns=col_names)

test_mm = mm_scaler.fit_transform(test)
test = pd.DataFrame(test_mm, columns=col_names)

X_train, X_test, y_train, y_test = train_test_split(train, train_labels, test_size=0.33)

In [6]:
## Run base models on full data ##
# comp_LR(X_train, X_test, y_train.values.ravel(), y_test.values.ravel())
# comp_SVM(X_train, X_test, y_train.values.ravel(), y_test.values.ravel())
# comp_NB(X_train, X_test, y_train.values.ravel(), y_test.values.ravel())
# comp_KNN(X_train, X_test, y_train.values.ravel(), y_test.values.ravel())
# comp_dtree(X_train, X_test, y_train.values.ravel(), y_test.values.ravel())

#Best is LR with validated AUC of ~0.876 - takes a REALLY long time

#Since data so large won't run more complex models on it

In [22]:
## Reducing dimensions - used random forest to get variable importance ##
#Don't use above split here - not prediction just for importance over all vars
clf = RandomForestClassifier(n_estimators = 1000, max_depth=10, random_state=0)
clf.fit(train, train_labels.values.ravel())

feature_importances = list(clf.feature_importances_)
columns = list(train.columns)

## Change n to modify how many features
#Found 100 was good with trade-off of complexity and accuracy
n = 100 #200, 300 overfit, 50 underfit, 100, 150 similar
top_n_idx = np.argsort(feature_importances)[-n:]
top_n_values = [feature_importances[i] for i in top_n_idx]

important_features = []
for i in list(top_n_idx):
    important_features.append(columns[i])

#Use above row splits and subset at columns
rf_train = X_train[important_features]
rf_test = X_test[important_features]

In [52]:
## Use reduced dataset on models ##
# Run base models - will print results automatically
comp_LR(rf_train, rf_test, y_train.values.ravel(), y_test.values.ravel())

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Best LR model Index: 0, AUC: 0.8836045836852348


In [13]:
comp_SVM(rf_train, rf_test, y_train.values.ravel(), y_test.values.ravel())

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Best SVM model Index: 0, AUC: 0.711165067292274


In [14]:
comp_NB(rf_train, rf_test, y_train.values.ravel(), y_test.values.ravel())

Best NB model Index: 0, AUC: 0.7894145079022004


In [15]:
comp_KNN(rf_train, rf_test, y_train.values.ravel(), y_test.values.ravel())

Best KNN model Index: 3, AUC: 0.7974828646008106


In [16]:
comp_dtree(rf_train, rf_test, y_train.values.ravel(), y_test.values.ravel())

Best DTree model Index: 0, AUC: 0.8777921835893869


In [20]:
#Now let's run on some ensemble methods
comp_rf(rf_train, rf_test, y_train.values.ravel(), y_test.values.ravel())

Best RF Num_Est: 200, depth:9,  AUC: 0.8861452663594408


In [21]:
comp_ada(rf_train, rf_test, y_train.values.ravel(), y_test.values.ravel())

Best Adaboost Num_Est: 17, AUC: 0.8887039184665846


In [22]:
NN_norm_opt(rf_train, rf_test, y_train, y_test)

(0.865860276844727, 'rmsprop')

In [44]:
NN_valid(rf_train, rf_test, y_train, y_test, opt="rmsprop")

Best NN Batch: 25, Neurons: 30, Layers: 2, AUC: 0.8685987985148234


In [7]:
## PCA to reduce dimenstions then run same models ##
pca = PCA(0.75, svd_solver="full")
#Want to use scaled data here
pca.fit(train)
p_train = pca.transform(train)
p_test = pca.transform(test)

In [8]:
pca_train, pca_test, ytrain, ytest = train_test_split(p_train, train_labels, test_size=0.33)

In [9]:
## Use reduced PCA dataset on models ##
# Run base models - will print results automatically
comp_LR(pca_train, pca_test, ytrain.values.ravel(), ytest.values.ravel())

Best LR model Index: 3, AUC: 0.8190707661172979


In [10]:
comp_SVM(pca_train, pca_test, ytrain.values.ravel(), ytest.values.ravel())

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Best SVM model Index: 1, AUC: 0.6346257676471458


In [11]:
comp_NB(pca_train, pca_test, ytrain.values.ravel(), ytest.values.ravel())

Best NB model Index: 2, AUC: 0.7886630868332601


In [12]:
comp_KNN(pca_train, pca_test, ytrain.values.ravel(), ytest.values.ravel())

Best KNN model Index: 3, AUC: 0.7345969021310246


In [13]:
comp_dtree(pca_train, pca_test, ytrain.values.ravel(), ytest.values.ravel())

Best DTree model Index: 0, AUC: 0.70259054391379


In [14]:
#Now let's run on some ensemble methods
comp_rf(pca_train, pca_test, ytrain.values.ravel(), ytest.values.ravel())

Best RF Num_Est: 190, depth:9,  AUC: 0.7806430498893013


In [40]:
comp_ada(pca_train, pca_test, ytrain.values.ravel(), ytest.values.ravel())

Best Adaboost Num_Est: 17, AUC: 0.7925930158944863


In [38]:
#Tune NN and run
NN_norm_opt(pca_train, pca_test, ytrain, ytest) #Best was rmsprop

(0.8172319408844636, 'rmsprop')

In [45]:
NN_valid(pca_train, pca_test, ytrain, ytest, opt="rmsprop")

Best NN Batch: 20, Neurons: 30, Layers: 2, AUC: 0.7623890047339121


In [ ]:
#Do two stacked models with subset data - one with two best models and one with three best models
est_3 = [('rf', RandomForestClassifier(n_estimators=200, max_depth = 9)),
         ('ada', AdaBoostClassifier(n_estimators=17)), 
         ('lg', LogisticRegression(solver='saga', penalty="l1", max_iter=100))]
est_2 = [('rf', RandomForestClassifier(n_estimators=100, max_depth = 10)),
         ('ada', AdaBoostClassifier(n_estimators=100))]

In [41]:
stack(rf_train, rf_test, y_train.values.ravel(), y_test.values.ravel(), ests=est_3)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

AUC of stacked: 0.8902899445248642


In [42]:
stack(rf_train, rf_test, y_train.values.ravel(), y_test.values.ravel(), ests=est_2)

AUC of stacked: 0.8861130270826999


In [41]:
## Use top 2 best models to get probability of classification to submit ##
#Want probability because using AUC to compute
#Stacked with reduced variable importance
top1 = StackingClassifier(estimators = est_3)
top1.fit(train[important_features], train_labels.values.ravel())
prob_top1 = top1.predict_proba(test[important_features])
pred_top1 = prob_top1[:,1]

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

In [32]:
pred_top1 = pd.DataFrame(pred_top1)
pred_top1['id'] = test_id
pred_top1.to_csv("top_prediction.csv")

In [34]:
top2 =  AdaBoostClassifier(n_estimators=17)
top2.fit(train[important_features], train_labels.values.ravel())
prob_top2 = top2.predict_proba(test[important_features])
pred_top2 = prob_top2[:,1]
pred_top2 = pd.DataFrame(pred_top2)
pred_top2['id'] = test_id
pred_top2.to_csv("top2_prediction.csv")

In [35]:
pred_top1

,0,id
0,0.094222,14061
1,0.192832,16467
2,0.804983,25725
3,0.613568,9100
4,0.505400,32597
...,...,...
9240,0.868917,31430
9241,0.437476,3876
9242,0.839335,20710
9243,0.724148,26015


In [36]:
pred_top2

,0,id
0,0.472904,14061
1,0.469834,16467
2,0.534714,25725
3,0.519413,9100
4,0.515126,32597
...,...,...
9240,0.533943,31430
9241,0.504187,3876
9242,0.509907,20710
9243,0.513823,26015
